An outline of the ultimate data processing to create usable master dataframes from which models can be easily built. As with other steps in this process, I did not run this at scale out of a notebook like, rather I have a .py file that I ran at the terminal, this notebook is solely for demonstration purposes. Some of these steps take a while to run and performing out of a notebook like this is inefficient: it made more sense to do the processing, save down the results as CSVs and then load those into a seperate notebook to visualize and build models.

In [264]:
import pandas as pd
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime
import sklearn

First, and easiest step was to read in previously saved down stock info. The stock api I used was simple and quick enough that I could have called newly updated stock values on the fly without really any loss of speed, but I already had saved down all the historical stock data locally.

I altered this to function later on to bring in empty columns (or columns filled with zeros) to correspond with the sentiment values from the NYT and twitter - the logic of this will become clear later on.

In [51]:
def stock_formatter_lite(stock_ticker):
    #takes in the stock ticker, since all csvs follow same naming conventions
    stock_df = pd.read_csv(f'stock_data/{stock_ticker}.csv')
    stock_df['Unnamed: 0'] = pd.to_datetime(stock_df['Unnamed: 0'])
    stock_df.set_index('Unnamed: 0',inplace=True)
    # And here are six columns which will later be filled with sentiment values
    stock_df['nyt_compound'] = [0]*len(stock_df)
    stock_df['nyt_pos'] = [0]*len(stock_df)
    stock_df['nyt_neg'] = [0]*len(stock_df)
    stock_df['twitter_compound'] = [0]*len(stock_df)
    stock_df['twitter_pos'] = [0]*len(stock_df)
    stock_df['twitter_neg'] = [0]*len(stock_df)
    return stock_df

In [196]:
tesla_stock = stock_formatter_lite('TSLA')

In [53]:
tesla_stock.tail()

,1. open,2. high,3. low,4. close,5. volume,nyt_compound,nyt_pos,nyt_neg,twitter_compound,twitter_pos,twitter_neg
Unnamed: 0,,,,,,,,,,,
2010-07-06,20.00,20.0000,15.83,16.11,6866900,0,0,0,0,0,0
2010-07-02,23.00,23.1000,18.71,19.20,5139800,0,0,0,0,0,0
2010-07-01,25.00,25.9200,20.27,21.96,8218800,0,0,0,0,0,0
2010-06-30,25.79,30.4192,23.30,23.83,17187100,0,0,0,0,0,0
2010-06-29,19.00,25.0000,17.54,23.89,18766300,0,0,0,0,0,0


And now to create dataframes with new york times sentiment data. First, the dataframe cleaning function and term counting helper function I previously used in the nyt exploration notebook.

Next comes a function to take in a series of file names and compile a master dataframe from them.

In [38]:
def term_counter(string, word):
    tokens = nltk.word_tokenize(string)
    return nltk.FreqDist(word.lower() for word in tokens)[word]

def dataframe_cleaner(df, term, sensitivity):
    indexes_to_drop = []
    for n in range(0,len(df)):
        try:
            if term_counter(df.iloc[n]['text'],term) < sensitivity:
                indexes_to_drop.append(n)
        except:
            indexes_to_drop.append(n)
    df.drop(df.index[indexes_to_drop],axis=0,inplace=True)
    df['compound'] = [sid.polarity_scores(text)['compound'] for text in df['text']]
    df['neg'] = [sid.polarity_scores(text)['neg'] for text in df['text']]
    df['pos'] = [sid.polarity_scores(text)['pos'] for text in df['text']]
    return df

In [143]:
def nyt_sentiment(sentiment_files, company_name, sensitivity):
    dataframes=[]
    for file in sentiment_files:
        df = pd.read_csv(f'nyt_text/{file}.csv')
        df = dataframe_cleaner(df, company_name, sensitivity)
        dataframes.append(df)
    master_df = pd.concat(dataframes)
    #Also, making the dataframe indexed to the date, properly formatted as datetime
    master_df['date'] = pd.to_datetime(master_df['date']) 
    #Occasionally there may be more than one nyt article on a given topic in a given day
    #I simply average the sentiment values across them in that instance
    master_df = master_df.groupby('date').mean()
    master_df.drop('Unnamed: 0',axis=1,inplace=True)
    return master_df

In [144]:
files = ['tesla_2016','tesla_2017','tesla_2018']
tesla_nyt = nyt_sentiment(files ,'tesla', 2)

In [145]:
tesla_nyt.head()

,compound,neg,pos
date,,,
2016-01-14,0.96750,0.0520,0.079
2016-01-17,0.99140,0.0260,0.072
2016-03-02,-0.99420,0.1200,0.044
2016-04-01,0.97015,0.0195,0.064
2016-04-04,0.99740,0.0500,0.108


And now something similar with twitter. Here's a function which takes in the file names and returns a master dataframe.

In [55]:
def twitter_sentiment(sentiment_files):
    dataframes = []
    for file in sentiment_files:
        df = pd.read_csv(f'twitter/{file}.csv')
        df.drop('Unnamed: 0',axis=1,inplace=True)
        dataframes.append(df)
    master_df = pd.concat(dataframes)
    master_df['date'] = pd.to_datetime(master_df['date'])
    master_df.set_index('date', inplace=True)
    return master_df

In [58]:
files = ['Tesla_twitter_sentiment_2016','Tesla_twitter_sentiment_2017','Tesla_twitter_sentiment_2018']
tesla_twitter = twitter_sentiment(files)
tesla_twitter.head()

,neg,pos,compound
date,,,
2016-01-01,0.054,0.128,0.9684
2016-01-02,0.061,0.081,0.7316
2016-01-03,0.044,0.016,-0.4696
2016-01-04,0.034,0.087,0.9878
2016-01-05,0.020,0.169,0.9983


And now to the real challenge, combining these dataframes into a master in a way that overcame a couple of date issues and gave you something workable.

The biggest issue here was that the dates didn't really align between the three dataframes:
1. Stock data is only avalaible for days the market is open (obvioiusly): no weekends ore holidays in the stock dataframe.
2. NYT sentiment is only available on days when there happened to be at least one article (again, obviously). Some companies had articles about them more often than others.
3. Twitter sentiment comes almost every day a year (there were only ever a handful of days without data out of the whole range)

How do we handle the mismatch in dates. The first key point is that we can't actually resample the stock data to be daily - it only really makes sense to talk about the price movement between days the market is open. So, my goal is clearly to pull the sentiment values from the other data frames into the stock one. The other key thing to note is that there's a lot of sentiment data coming on days when the market isn't open - plenty of articles are published on the weekends and twitter never sleeps. I clearly want to caputre this sentiment somehow.

My function here to clean up and create the master dataframe handles this by averaging all of the sentiment values from the off-days together. So, my model is going to, say, try to predict price movement starting on Friday from the sentiment on Thursday. For Monday, which is preceded by days the market is closed, we'll be using the averaged sentiment values from Friday through Sunday. Since days the market is closed come at semi-irregular intervals (due to holidays) the function has to go through a little loop to find out exactly how many days need to be averaged together in any given case.

In [274]:
#Little helper function that simply returns the amount of price movement between two days
def price_change(stock_df,index, n_days):
    date = stock_df.index[index]
    date_str = str(date)[:10]
    future = str(stock_df.index[index-n_days])[:10]
    initial_price = stock_df.loc[date_str,'4. close'][0]
    later_price = stock_df.loc[future,'4. close'][0]
    return (later_price/initial_price) -1

def fill_out_sentiments(stock_df,nyt_df,twitter_df):
    # stock csvs have data going back to 2010, so starting from 2016
    stock_df = stock_df[:'2016']
    for date in stock_df.index[1:]:
        print(date) #so I can keep track when it runs in terminal
        #first check if the following day was also a market open day
        #if so, values can be pulled directly in
        if date + datetime.timedelta(1) in stock_df.index:
            date_str = str(date)[:10] #pandas indexing requires the string, not the timestamp
            if date_str in nyt_df.index:
                stock_df.loc[date_str,'nyt_compound'] = nyt_df.loc[date_str,'compound']
                stock_df.loc[date_str,'nyt_pos'] = nyt_df.loc[date_str,'pos']
                stock_df.loc[date_str,'nyt_neg'] = nyt_df.loc[date_str,'neg']
            if date_str in twitter_df.index:
                stock_df.loc[date_str,'twitter_compound'] = twitter_df.loc[date_str,'compound']
                stock_df.loc[date_str,'twitter_pos'] = twitter_df.loc[date_str,'pos']
                stock_df.loc[date_str,'twitter_neg'] = twitter_df.loc[date_str,'neg']
        #Now, if the market was closed the following day, when was the next day the market was open?
        else:
            n = 1
            while date + datetime.timedelta(n) not in stock_df.index:
                n+=1
            #n is now the number of days until the next open day
            span_end = str(date + datetime.timedelta(n))[:10] #now we have a range of days to check
            date_str = str(date)[:10]
            #And we're going to average all the values from days in that range
            stock_df.loc[date_str,'nyt_compound'] = nyt_df.loc[date_str:span_end,'compound'].mean()
            stock_df.loc[date_str,'nyt_pos'] = nyt_df.loc[date_str:span_end,'pos'].mean()
            stock_df.loc[date_str,'nyt_neg'] = nyt_df.loc[date_str:span_end,'neg'].mean()
            stock_df.loc[date_str,'twitter_compound'] = twitter_df.loc[date_str:span_end,'compound'].mean()
            stock_df.loc[date_str,'twitter_pos'] = twitter_df.loc[date_str:span_end,'pos'].mean()
            stock_df.loc[date_str,'twitter_neg'] = twitter_df.loc[date_str:span_end,'neg'].mean()
        #This pulls in the occasional NaN for days with no sentiment, fill those with 0
        stock_df.fillna(0,inplace=True)
        stock_df.index.rename('date',inplace=True)
        #Also adding in the price change columns here
        stock_df['day_change'] = [0]+[price_change(stock_df,n,1) for n in range(1,len(stock_df))]
        stock_df['three_day'] = 3*[0]+[price_change(stock_df,n,1) for n in range(3,len(stock_df))]
        stock_df['week_change'] = 5*[0]+[price_change(stock_df,n,1) for n in range(5,len(stock_df))]
    return stock_df

In [275]:
tesla_master = fill_out_sentiments(tesla_stock,tesla_nyt,tesla_twitter)

2019-07-18 00:00:00


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

2019-07-17 00:00:00
2019-07-16 00:00:00
2019-07-15 00:00:00
2019-07-12 00:00:00
2019-07-11 00:00:00
2019-07-10 00:00:00
2019-07-09 00:00:00
2019-07-08 00:00:00
2019-07-05 00:00:00
2019-07-03 00:00:00
2019-07-02 00:00:00
2019-07-01 00:00:00
2019-06-28 00:00:00
2019-06-27 00:00:00
2019-06-26 00:00:00
2019-06-25 00:00:00
2019-06-24 00:00:00
2019-06-21 00:00:00
2019-06-20 00:00:00
2019-06-19 00:00:00
2019-06-18 00:00:00
2019-06-17 00:00:00
2019-06-14 00:00:00
2019-06-13 00:00:00
2019-06-12 00:00:00
2019-06-11 00:00:00
2019-06-10 00:00:00
2019-06-07 00:00:00
2019-06-06 00:00:00
2019-06-05 00:00:00
2019-06-04 00:00:00
2019-06-03 00:00:00
2019-05-31 00:00:00
2019-05-30 00:00:00
2019-05-29 00:00:00
2019-05-28 00:00:00
2019-05-24 00:00:00
2019-05-23 00:00:00
2019-05-22 00:00:00
2019-05-21 00:00:00
2019-05-20 00:00:00
2019-05-17 00:00:00
2019-05-16 00:00:00
2019-05-15 00:00:00
2019-05-14 00:00:00
2019-05-13 00:00:00
2019-05-10 00:00:00
2019-05-09 00:00:00
2019-05-08 00:00:00
2019-05-07 00:00:00


2017-11-27 00:00:00
2017-11-24 00:00:00
2017-11-22 00:00:00
2017-11-21 00:00:00
2017-11-20 00:00:00
2017-11-17 00:00:00
2017-11-16 00:00:00
2017-11-15 00:00:00
2017-11-14 00:00:00
2017-11-13 00:00:00
2017-11-10 00:00:00
2017-11-09 00:00:00
2017-11-08 00:00:00
2017-11-07 00:00:00
2017-11-06 00:00:00
2017-11-03 00:00:00
2017-11-02 00:00:00
2017-11-01 00:00:00
2017-10-31 00:00:00
2017-10-30 00:00:00
2017-10-27 00:00:00
2017-10-26 00:00:00
2017-10-25 00:00:00
2017-10-24 00:00:00
2017-10-23 00:00:00
2017-10-20 00:00:00
2017-10-19 00:00:00
2017-10-18 00:00:00
2017-10-17 00:00:00
2017-10-16 00:00:00
2017-10-13 00:00:00
2017-10-12 00:00:00
2017-10-11 00:00:00
2017-10-10 00:00:00
2017-10-09 00:00:00
2017-10-06 00:00:00
2017-10-05 00:00:00
2017-10-04 00:00:00
2017-10-03 00:00:00
2017-10-02 00:00:00
2017-09-29 00:00:00
2017-09-28 00:00:00
2017-09-27 00:00:00
2017-09-26 00:00:00
2017-09-25 00:00:00
2017-09-22 00:00:00
2017-09-21 00:00:00
2017-09-20 00:00:00
2017-09-19 00:00:00
2017-09-18 00:00:00


2016-04-13 00:00:00
2016-04-12 00:00:00
2016-04-11 00:00:00
2016-04-08 00:00:00
2016-04-07 00:00:00
2016-04-06 00:00:00
2016-04-05 00:00:00
2016-04-04 00:00:00
2016-04-01 00:00:00
2016-03-31 00:00:00
2016-03-30 00:00:00
2016-03-29 00:00:00
2016-03-28 00:00:00
2016-03-24 00:00:00
2016-03-23 00:00:00
2016-03-22 00:00:00
2016-03-21 00:00:00
2016-03-18 00:00:00
2016-03-17 00:00:00
2016-03-16 00:00:00
2016-03-15 00:00:00
2016-03-14 00:00:00
2016-03-11 00:00:00
2016-03-10 00:00:00
2016-03-09 00:00:00
2016-03-08 00:00:00
2016-03-07 00:00:00
2016-03-04 00:00:00
2016-03-03 00:00:00
2016-03-02 00:00:00
2016-03-01 00:00:00
2016-02-29 00:00:00
2016-02-26 00:00:00
2016-02-25 00:00:00
2016-02-24 00:00:00
2016-02-23 00:00:00
2016-02-22 00:00:00
2016-02-19 00:00:00
2016-02-18 00:00:00
2016-02-17 00:00:00
2016-02-16 00:00:00
2016-02-12 00:00:00
2016-02-11 00:00:00
2016-02-10 00:00:00
2016-02-09 00:00:00
2016-02-08 00:00:00
2016-02-05 00:00:00
2016-02-04 00:00:00
2016-02-03 00:00:00
2016-02-02 00:00:00


In [278]:
tesla_master.tail(10)

,1. open,2. high,3. low,4. close,5. volume,nyt_compound,nyt_pos,nyt_neg,twitter_compound,twitter_pos,twitter_neg,day_change,three_day,week_change
date,,,,,,,,,,,,,,
2016-01-15,198.97,205.0700,197.25,204.99,5578640,0.9914,0.072,0.026,0.988160,0.12720,0.01800,-0.001317,-0.001317,-0.001317
2016-01-14,202.21,210.0000,193.38,206.18,6490741,0.9675,0.079,0.052,0.995600,0.13300,0.02400,-0.005772,-0.005772,-0.005772
2016-01-13,212.01,212.6500,200.00,200.31,4126416,0.0000,0.000,0.000,0.995800,0.12500,0.00900,0.029305,0.029305,0.029305
2016-01-12,211.60,213.7395,205.31,209.97,3091917,0.0000,0.000,0.000,0.975900,0.06900,0.00900,-0.046007,-0.046007,-0.046007
2016-01-11,214.01,214.4500,203.00,207.85,4091422,0.0000,0.000,0.000,0.996200,0.10800,0.02500,0.010200,0.010200,0.010200
2016-01-08,217.86,220.4400,210.77,211.00,3628058,0.0000,0.000,0.000,0.993775,0.14575,0.03275,-0.014929,-0.014929,-0.014929
2016-01-07,214.19,218.4400,213.67,215.65,3554251,0.0000,0.000,0.000,0.997700,0.17000,0.03400,-0.021563,-0.021563,-0.021563
2016-01-06,220.00,220.0500,215.98,219.04,3779128,0.0000,0.000,0.000,0.996700,0.18200,0.01600,-0.015477,-0.015477,-0.015477
2016-01-05,226.36,226.8900,220.00,223.43,3186752,0.0000,0.000,0.000,0.998300,0.16900,0.02000,-0.019648,-0.019648,-0.019648


In [279]:
tesla_master.to_csv('tesla_master.csv')

When I ran this from the terminal I wrapped everything into one final function which would take in simply the lists of file names, the company name and sensitivity, perform all the intermediate steps and finally save the result down as a well formatted csv:

In [ ]:
def complete_dataframe_creator(ticker,nyt_files,company_name,twitter_files,sensitivity):
    stock_df = stock_formatter_lite(ticker)
    nyt_df = nyt_sentiment(nyt_files, company_name, sensitivity)
    twitter_df = twitter_sentiment(twitter_files)
    master_df = fill_out_sentiments(stock_df,nyt_df,twitter_df)
    master_df.to_csv(f'{company_name}_master.csv')
    pass